    This file is a part of the lor_neuro_rat project.
    Copyright (C) 2019 anonimous

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

    Please contact with me by E-mail: shkolnick.kun@gmail.com

# Подготовка удаленных сообщений и причин удаления для разметки.

In [1]:
import pandas as pd
#Загрузка
df = pd.read_pickle('data/all_data_as_dataframe.pkl')
#Сортировка
df.sort_values(by=['DelReason'], inplace=True, ascending=False)
df.reset_index(inplace=True, drop=True)

print(df.head())

                         TopId             MsgId                     Creator  \
0  /forum/development/11896689  comment-11896982  /people/bookman900/profile   
1        /forum/talks/12648516  comment-12649855     /people/mandala/profile   
2      /forum/desktop/11530019  comment-11530154       /people/zzdnx/profile   
3      /forum/desktop/12261887  comment-12281227       /people/zzdnx/profile   
4      /forum/desktop/11018384  comment-11025530       /people/zzdnx/profile   

                            Time                                   SrcLink  \
0  2015-08-27T12:33:42.308+03:00  /forum/development/11896689?cid=11896705   
1  2016-06-07T20:01:40.393+03:00        /forum/talks/12648516?cid=12649823   
2  2015-04-20T16:37:05.253+03:00      /forum/desktop/11530019?cid=11530137   
3  2016-01-18T16:18:38.985+03:00      /forum/desktop/12261887?cid=12281126   
4  2014-11-11T15:22:21.668+03:00      /forum/desktop/11018384?cid=11025521   

                                           DelReas

In [2]:
import re
#Удаленные сообщения
DeletedMsg = df[df['DelReason'] != '']
#Создатель
Creator = DeletedMsg['Creator'].apply(lambda x: x.split('/')[2] if x else x)
#Кто удалил
DeletedBy = DeletedMsg['DelReason'].apply(lambda x: re.sub('Сообщение удалено ', '', x).split()[0] if x else '')
#Удаленные модераторами
DeletedByMod = DeletedMsg[Creator != DeletedBy]

In [3]:
DeletedByMod.head()


,TopId,MsgId,Creator,Time,SrcLink,DelReason,Txt,Code,Quotes
0,/forum/development/11896689,comment-11896982,/people/bookman900/profile,2015-08-27T12:33:42.308+03:00,/forum/development/11896689?cid=11896705,"Сообщение удалено. Причина: 4.1, 5.2",[Учетверяю!],[],[]
1,/forum/talks/12648516,comment-12649855,/people/mandala/profile,2016-06-07T20:01:40.393+03:00,/forum/talks/12648516?cid=12649823,Сообщение удалено по причине Национальные/рели...,[Уровень развития цивилизации всего лишь. На н...,[],[]
539,/forum/development/12311646,comment-12311699,,2016-01-29T17:14:41.148+03:00,/forum/development/12311646?cid=12311687,Сообщение удалено yangramovich по причине Уже ...,"[Ок,и что делать?]",[],[]
540,/forum/development/12311646,comment-12312131,,2016-01-29T19:30:55.023+03:00,/forum/development/12311646?cid=12311871,Сообщение удалено yangramovich по причине Ошиб...,[Попробовал.На русском выводятся квадраты с чи...,[],[]
541,/forum/games/12223342,comment-12224327,,2015-12-26T12:40:58.952+03:00,,Сообщение удалено yangramovich по причине Не д...,"[UPD1(Решение проблемы):\n, Ставим блоб для ви...",[],[]


In [4]:
MsgOfInterest = DeletedByMod.copy()

In [5]:
print(len(MsgOfInterest))
for r in MsgOfInterest['DelReason']:
    if 'по причине' not in r:
        print(r)

91311
Сообщение удалено. Причина: 4.1, 5.2


Всего одно сообщение, ну и ладно

In [6]:
MsgOfInterest = MsgOfInterest[MsgOfInterest['DelReason'].apply(lambda x: 'по причине' in x)].copy()

In [7]:
Reason = MsgOfInterest['DelReason'].apply(lambda x: x.split('по причине')[1])

In [8]:
for r in MsgOfInterest['DelReason']:
    try:
        a = r.split('по причине')[1].split()[-1].split('(')[1]
    except Exception as e:
        print(e)
        print(r)

list index out of range
Сообщение удалено по причине Национальные/религиозные/политические споры.


In [9]:
def get_score(x):
    try:
        score = int(x.split('по причине')[1].split()[-1].split('(')[1].split(')')[0])
    except Exception as e:
        print(e)
        print(x)
        score = 0
    return score

Score = MsgOfInterest['DelReason'].apply(get_score)

list index out of range
Сообщение удалено по причине Национальные/религиозные/политические споры.


In [15]:
#Описания причин удаления, дальше будем размечать lookup table вручную
import re
"""
reasons = ['7.1 Ответ на некорректное сообщение',
           '3.2 Неверная кодировка',
           '3.3 Некорректное форматирование',
           '3.4 Пустое сообщение',
           '4.2 Вызывающе неверная информация']
"""

reasons = []
for r in Reason:
    r = re.sub(r',', ';', r)
    if r not in reasons:
        reasons.append(r)
reasons.sort()

reason_lut = pd.DataFrame({'Reason':reasons, 'Class':[0]*len(reasons)})
reason_lut.to_csv('data/markup/reasons_lut.csv', index=False, sep=',')

In [16]:
print(len(MsgOfInterest))
print(len(Reason))
print(len(Score))

91310
91310
91310


In [17]:
MsgOfInterest['Reason'] = Reason.values
MsgOfInterest['Score'] = Score.values
MsgOfInterest.to_pickle('data/MsgOfInterestDataFrame.pkl')